In [1]:
# Import libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dt_ids7_export_utils as bh_utils

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'IDS7 RRH XA.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH XA reduced.xlsx')

del path

# Remove unnessecary columns:
df_ids7 = bh_utils.remove_unnecessary_columns(df_ids7, True)
# Check for missing time data:
df_ids7 = bh_utils.filter_NaT(df_ids7, True)
# Check for cancelled procedures:
df_ids7 = bh_utils.filter_cancelled(df_ids7, True)
# Check for non-human subjects (e.g. phantoms, animals, etc.):
df_ids7 = bh_utils.filter_phantom_etc(df_ids7, True)
# Check for invalid accession numbers:
df_ids7 = bh_utils.check_accession_format(df_ids7, True)
# Check whether an accession number is in the DoseTrack data:
df_ids7 = bh_utils.check_accession_ids7_vs_dt(df_ids7, df_dt, True)
# Check whether examination has duplicated accession numbers that should be overwritten:
df_ids7 = bh_utils.overwrite_duplicated_accession_numbers(df_ids7, df_dt, True)

Dropping unnecessary column: Prioritet- og lesemerkeikon
Dropping unnecessary column: Lagt til i demonstrasjon-ikon
Dropping unnecessary column: Status
Number of rows with NaT in the column "Bestilt dato og tidspunkt": 1
Number of non-human subjects: 9
Number of rows with invalid accession number: 0
Number of accession numbers in IDS7: 2533
Number of accession numbers in IDS7 not in DoseTrack: 446
Inserted accession number: NRRH000008305818 for patient: PAS0452, time: 2023-07-03 00:00:00, accession numbers: ['NRRH000008305929' 'NRRH000008305818']
Inserted accession number: NRRH000008244732 for patient: PAS0607, time: 2023-05-18 13:00:00, accession numbers: ['NRRH000008244733' 'NRRH000008244732']
Inserted accession number: NRRH000007967678 for patient: PAS1790, time: 2023-01-03 07:00:00, accession numbers: ['NRRH000007967678' 'NRRH000007967670']
The accession numbers have been changed. Running check_accession_ids7_vs_dt
Number of accession numbers in IDS7: 2530
Number of accession numbe

In [3]:
len(df_ids7[(df_ids7['Avbrutt'] == 'Avbrutt') & df_ids7['Henvisning_i_dt'] == True])
avbrutt = df_ids7[(df_ids7['Avbrutt'] == 'Avbrutt') & df_ids7['Henvisning_i_dt'] == True]

In [4]:
bh_utils.export_examination_codes_to_text_file(df_ids7, 'RRH_XA1')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'RRH_XA2')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'RRH_XA4')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'RRH_RF2')
bh_utils.export_examination_codes_to_text_file(df_ids7, 'IRH_XA7')

We need to create logic for merging lines in the ids7 data:

During a booking the patient gets an accession number:
As the procedure moves forward more codes can be added to the same accession number. These codes should be merged into one line.

Check if the accession number exists in Dosetrack. If it does, check if there are any accession numbers within 12 hrs on the same patient on the same lab that does not have a corresponding entry in dosetrack. If so they might be in the same procedure.

Alternatively, if there are an accession number that is not included in doseTrack, check if there are any accession numbers within 12 hrs on the same patient on the same lab that does have a corresponding entry in dosetrack. If so they might be in the same procedure.

If neither of the procedures exists in dosetrack they might or might not be the same procedure.

Print a list of all the changes performed on the data.

In [ ]:
# Check for patients with multiple bookings on the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_time_with_different_accession(df_ids7)

In [ ]:
# Check for patients with multiple bookings on the same day but not the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_day_with_different_accession(df_ids7)